This notebook contains the initial experimentation code for generating the final summary of articles retrieved for a given image using the bing visual search api. This is done for the test data (first 1k samples). These summaries will be stored to provide initial context for the debating models.

In [1]:
import sys
sys.path.append("..")

import http.client, urllib.parse
import requests
import json
import os.path

BASE_URI = "https://api.bing.microsoft.com/v7.0/images/visualsearch"
SUBSCRIPTION_KEY = 'fcbb57be03f64cf191aeb1ba7835fc04'

from utils.data import get_data
from utils.external_retrieval import get_summary, get_webpage_text

In [2]:
def get_urls(file):
    matching_urls = []
    # Construct the request headers
    headers = {
        "Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY
    }

    # Send the POST request
    response = requests.post(
        BASE_URI,
        headers=headers,
        files=file
    )

    # Check if the request was successful
    if response.status_code == 200:
        # Process the response data
        data = response.json()
        #print(data)
        result_list = []
        if len(data['tags'][0]['actions'][0]['data']['value']) == 0:
            result_list = data['tags'][0]['actions'][2]['data']['value']
        else:
            result_list = data['tags'][0]['actions'][0]['data']['value']
        for res in result_list:
            if len(matching_urls) == 3:
                break
            else:
                matching_urls.append(res['hostPageUrl'])
        return matching_urls
    else:
        # Handle the error
        return matching_urls
        #print(f"Error: {response.status_code} - {response.text}")

In [44]:
for i in range(1):
    image, caption, image_path, annotation = get_data(i)
    file = {'image' : ('myfile', open(image_path, 'rb'))}
    matching_urls = get_urls(file)
    print("matching_urls: ", matching_urls)
    search_result = get_summary(matching_urls)

matching_urls:  ['https://www.theconstructionindex.co.uk/news/view/mps-call-for-government-action-on-blacklisting', 'https://socialistworker.co.uk/news/tens-of-thousands-more-names-could-have-been-on-blacklists/', 'https://www.theguardian.com/law/2021/may/27/recruiters-urged-to-clamp-down-on-exploitation-via-umbrella-companies']


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/homes/55/kumud/anaconda/envs/experiments/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Found 3 search results.


In [3]:
matching_urls = ['https://www.theconstructionindex.co.uk/news/view/mps-call-for-government-action-on-blacklisting', 'https://socialistworker.co.uk/news/tens-of-thousands-more-names-could-have-been-on-blacklists/', 'https://www.theguardian.com/law/2021/may/27/recruiters-urged-to-clamp-down-on-exploitation-via-umbrella-companies']

In [4]:
search_result = get_summary(matching_urls)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/homes/55/kumud/anaconda/envs/experiments/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/homes/55/kumud/anaconda/envs/experiments/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Found 3 search results.


In [5]:
print(search_result)

SUMMARY:
                    - The House of Commons Scottish Affairs committee has been investigating the use of the Consulting Association by major contractors to prevent suspected "troublemakers" from getting jobs in the industry.
                    - The committee has described the practice as an "organized conspiracy" by big construction firms to discriminate against workers who raised legitimate grievances over health and safety and other industrial issues.
                    - The committee has received written evidence and held a formal committee hearing on the 3rd of July with Gail Cartmail of Unite the Union, who has given what they believe to be clear proof that blacklisting for trade union and health and safety activities has been going on within the Crossrail project.
                    - The Consulting Association was shut down by the Information Commissioner in 2009.
                    - The scale of the blacklist may be far larger than previously thought and goes wel